# SpaceCat

In [ ]:
import os
import anndata
import pandas as pd

from SpaceCat.preprocess import preprocess_table
from SpaceCat.features import SpaceCat

## Cell table preprocessing

In [ ]:
output_dir = ''
cell_table = pd.read_csv('')

In [ ]:
# Convert csv to anndata table
markers = ['ECAD', 'CK17', 'CD45', 'CD3', 'CD4', 'CD8', 'FOXP3', 'CD20', 'CD56', 'CD14', 'CD68',
           'CD163', 'CD11c', 'HLADR', 'HLA1', 'ChyTr', 'Calprotectin', 'FAP', 'SMA', 'Vim', 'Fibronectin',
           'Collagen1', 'CD31', 'PDL1','Ki67','GLUT1','CD45RO','CD69', 'PD1','CD57','TBET', 'TCF1',
           'CD45RB', 'TIM3','IDO', 'LAG3', 'CD38', 'Fe']

cell_data_cols = ['fov', 'label', 'cell_meta_cluster', 'cell_cluster', 'cell_cluster_broad', 'compartment', 'compartment_area', 'area', 'cell_size', 'centroid_dif', 'convex_area', 'convex_hull_resid', 
                 'eccentricity', 'equivalent_diameter', 'major_axis_equiv_diam_ratio', 'major_axis_length',
                 'major_minor_axis_ratio', 'minor_axis_length', 'nc_ratio', 'num_concavities', 'perim_square_over_area',
                 'perimeter']
centroid_cols = ['centroid-0', 'centroid-1']

adata = anndata.AnnData(cell_table.loc[:, markers])
adata.obs = pd.merge(cell_table.loc[:, cell_data_cols, ], on = ['fov'])
adata.obsm['spatial'] = cell_table.loc[:, centroid_cols].values

adata.write_h5ad(os.path.join(output_dir, 'adata', 'adata_table.h5ad'))

In [ ]:
# Preprocessing (only needs to be run once)
functional_marker_thresholds = [['Ki67', 0.002], ['CD38', 0.004], ['CD45RB', 0.001], ['CD45RO', 0.002],
                                ['CD57', 0.002], ['CD69', 0.002], ['GLUT1', 0.002], ['IDO', 0.001],
                                ['LAG3', 0.002], ['PD1', 0.0005], ['PDL1', 0.001],
                                ['HLA1', 0.001], ['HLADR', 0.001], ['TBET', 0.0015], ['TCF1', 0.001],
                                ['TIM3', 0.001], ['Vim', 0.002], ['Fe', 0.1]]

adata = anndata.read_h5ad(os.path.join(output_dir, 'adata', 'adata_table.h5ad'))
adata_processed = preprocess_table(adata, functional_marker_thresholds)

adata_processed.write_h5ad(os.path.join(output_dir, 'adata', 'adata_table_processed.h5ad'))

## Feature Generation

In [ ]:
# Initialize class
adata_processed = anndata.read_h5ad(os.path.join(output_dir, 'adata', 'adata_table_processed.h5ad'))

features = SpaceCat(adata_processed, image_column='fov', label_column='label', 
                    cluster_columns=['cell_cluster', 'cell_cluster_broad'], 
                    compartment_column='compartment')


# Generate features and save anndata table
adata_processed = features.run_spacecat()

adata_processed.write_h5ad(os.path.join(output_dir, 'adata', 'adata_table_processed.h5ad'))

## Save Output to CSV (optional)

In [ ]:
# Save finalized tables to csv 
os.makedirs(os.path.join(output_dir, 'SpaceCat'), exist_ok=True)

adata_processed.uns['combined_feature_data'].to_csv(os.path.join(output_dir, 'SpaceCat', 'combined_feature_data.csv'))
adata_processed.uns['combined_feature_data_filtered'].to_csv(os.path.join(output_dir, 'SpaceCat', 'combined_feature_data_filtered.csv'))
adata_processed.uns['feature_metadata'].to_csv(os.path.join(output_dir, 'SpaceCat', 'feature_metadata.csv'))
adata_processed.uns['excluded_features'].to_csv(os.path.join(output_dir, 'SpaceCat', 'excluded_features.csv'))